# Código original

## 1. Asignación de etiquetas en imagen girada

In [16]:
import cv2
import numpy as np
import os

# Función para rotar la imagen
def rotar_imagen(imagen, angulo):
    (h, w) = imagen.shape[:2]
    centro = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(centro, angulo, 1.0)
    cos = np.abs(M[0, 0])
    sen = np.abs(M[0, 1])
    new_w = int((h * sen) + (w * cos))
    new_h = int((h * cos) + (w * sen))
    M[0, 2] += (new_w / 2) - centro[0]
    M[1, 2] += (new_h / 2) - centro[1]
    imagen_rotada = cv2.warpAffine(imagen, M, (new_w, new_h))
    return imagen_rotada, M

# Función para rotar un polígono (caja delimitadora)
def rotar_poligono(puntos, M):
    ones = np.ones(shape=(len(puntos), 1))
    puntos_ones = np.hstack([puntos, ones])
    puntos_rotados = M.dot(puntos_ones.T).T
    return puntos_rotados[:, :2].astype(int)

# Función para convertir las coordenadas de Roboflow a un cuadrilátero
def convertir_a_cuadrilatero(xmin, xmax, ymin, ymax):
    return [
        [xmin, ymin],  # Esquina superior izquierda
        [xmax, ymin],  # Esquina superior derecha
        [xmax, ymax],  # Esquina inferior derecha
        [xmin, ymax]   # Esquina inferior izquierda
    ]

# Ruta a la imagen
ruta_imagen = 'ED6_10_sf11.png'

# Verificar si el archivo existe
if not os.path.isfile(ruta_imagen):
    print(f"Error: La imagen no se encuentra en la ruta especificada: {ruta_imagen}")
else:
    imagen = cv2.imread(ruta_imagen)
    
    if imagen is None:
        print(f"Error: No se pudo cargar la imagen.")
    else:
        print("Imagen cargada correctamente.")

        # Etiquetas proporcionadas por Roboflow
        # Cada objeto tiene coordenadas xmin, xmax, ymin, ymax
        #etiquetas = [
        #    {'xmin': 654, 'xmax': 657, 'ymin': 338, 'ymax': 344}
        #]
        etiquetas = etiquetas_imged6

        # Convertir las etiquetas a cuadriláteros
        cajas = [convertir_a_cuadrilatero(etiq['xmin'], etiq['xmax'], etiq['ymin'], etiq['ymax']) for etiq in etiquetas]
        print
        
        # Crear una copia de la imagen para dibujar las cajas
        imagen_con_cajas = imagen.copy()
        for caja in cajas:
            puntos = np.array(caja, np.int32)
            puntos = puntos.reshape((-1, 1, 2))
            cv2.polylines(imagen_con_cajas, [puntos], isClosed=True, color=(255, 0, 0), thickness=2)  # Azul para las cajas originales
        
        # Rotar la imagen
        angulo = 10  # grados
        imagen_rotada, M = rotar_imagen(imagen, angulo)
        print("Imagen rotada exitosamente.")
        
        # Rotar las cajas
        cajas_rotadas = [rotar_poligono(np.array(caja), M) for caja in cajas]
        print("Cajas rotadas exitosamente.")
        
        # Dibujar las cajas rotadas en la imagen rotada
        for caja_rotada in cajas_rotadas:
            caja_rotada = caja_rotada.reshape((-1, 1, 2))
            cv2.polylines(imagen_rotada, [caja_rotada], isClosed=True, color=(0, 255, 0), thickness=2)  # Verde para las cajas rotadas
            print(caja_rotada)
        
        # Redimensionar imágenes para que tengan la misma altura
        altura_original = imagen_con_cajas.shape[0]
        altura_rotada = imagen_rotada.shape[0]
        if altura_original != altura_rotada:
            ratio = altura_original / altura_rotada
            nuevo_ancho = int(imagen_rotada.shape[1] * ratio)
            imagen_rotada = cv2.resize(imagen_rotada, (nuevo_ancho, altura_original))
            cajas_rotadas = [ (caja_rotada * ratio).astype(int) for caja_rotada in cajas_rotadas]
            print(cajas_rotadas)
        
        # Concatenar las imágenes horizontalmente
        imagen_comparacion = np.hstack((imagen_con_cajas, imagen_rotada))
        
        # Añadir texto para identificar cada imagen
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(imagen_comparacion, 'Original', (10, 30), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(imagen_comparacion, 'Rotada', (imagen_con_cajas.shape[1] + 10, 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Guardar la imagen comparativa
        cv2.imwrite('ED6_10_sf11_v1.png', imagen_comparacion)


Imagen cargada correctamente.
Imagen rotada exitosamente.
Cajas rotadas exitosamente.
[[[1042  544]]

 [[1051  543]]

 [[1053  551]]

 [[1044  552]]]
[[[1055  533]]

 [[1059  532]]

 [[1061  540]]

 [[1056  541]]]
[[[1026  538]]

 [[1035  536]]

 [[1036  542]]

 [[1027  544]]]
[[[1006  525]]

 [[1013  524]]

 [[1014  530]]

 [[1008  531]]]
[[[1028  522]]

 [[1034  521]]

 [[1035  528]]

 [[1030  529]]]
[[[1041  506]]

 [[1047  505]]

 [[1047  510]]

 [[1042  511]]]
[[[1076  520]]

 [[1082  519]]

 [[1083  525]]

 [[1077  526]]]
[[[1054  580]]

 [[1060  579]]

 [[1061  587]]

 [[1055  588]]]
[[[1065  574]]

 [[1072  573]]

 [[1073  581]]

 [[1066  582]]]
[[[1069  568]]

 [[1076  567]]

 [[1077  572]]

 [[1070  573]]]
[[[1075  576]]

 [[1081  575]]

 [[1082  581]]

 [[1076  582]]]
[[[1086  560]]

 [[1094  559]]

 [[1095  566]]

 [[1087  568]]]
[[[1004  591]]

 [[1011  589]]

 [[1012  595]]

 [[1005  597]]]
[[[1011  573]]

 [[1018  572]]

 [[1019  579]]

 [[1012  580]]]
[[[1000  508]]

 [

# Prueba 3

## 1.1. Extraer coordenadas desde un XML

In [5]:
import xml.etree.ElementTree as ET

#cargar el archivo XML
ruta_xml = "data/train/2024-10-08T09_35_18-739305_1920x1080_0_00_00-000400_jpg.rf.8bfb6711a6b4814df45708073c0732e4.xml"

#parsear el archivo XML
tree = ET.parse(ruta_xml)
root = tree.getroot()

#lista para guardar las etiquetas
etiquetas_imged6 = []

#iterar sobre cada objeto en el archivo XML
for obj in root.findall('object'):
    #obtener las coordenadas xmin, xmax, ymin, ymax
    clase = str(obj.find('name').text)
    xmin = int(obj.find('bndbox/xmin').text)
    xmax = int(obj.find('bndbox/xmax').text)
    ymin = int(obj.find('bndbox/ymin').text)
    ymax = int(obj.find('bndbox/ymax').text)
    
    #agregar las coordenadas al array de etiquetas
    etiquetas_imged6.append({
        'class':clase,
        'xmin': xmin,
        'xmax': xmax,
        'ymin': ymin,
        'ymax': ymax
    })

#aarray de etiquetas
print(etiquetas_imged6)

[{'class': 'pellet', 'xmin': 313, 'xmax': 328, 'ymin': 615, 'ymax': 627}, {'class': 'pellet', 'xmin': 571, 'xmax': 578, 'ymin': 337, 'ymax': 347}, {'class': 'pellet', 'xmin': 390, 'xmax': 396, 'ymin': 616, 'ymax': 625}, {'class': 'pellet', 'xmin': 596, 'xmax': 609, 'ymin': 580, 'ymax': 587}, {'class': 'semipellet', 'xmin': 561, 'xmax': 570, 'ymin': 821, 'ymax': 831}, {'class': 'semipellet', 'xmin': 653, 'xmax': 662, 'ymin': 854, 'ymax': 865}, {'class': 'pellet', 'xmin': 287, 'xmax': 298, 'ymin': 634, 'ymax': 644}, {'class': 'semipellet', 'xmin': 276, 'xmax': 286, 'ymin': 656, 'ymax': 664}, {'class': 'semipellet', 'xmin': 291, 'xmax': 302, 'ymin': 657, 'ymax': 667}, {'class': 'semipellet', 'xmin': 238, 'xmax': 248, 'ymin': 657, 'ymax': 665}, {'class': 'semipellet', 'xmin': 262, 'xmax': 271, 'ymin': 636, 'ymax': 646}, {'class': 'semipellet', 'xmin': 529, 'xmax': 537, 'ymin': 392, 'ymax': 400}, {'class': 'semipellet', 'xmin': 665, 'xmax': 675, 'ymin': 474, 'ymax': 481}, {'class': 'semipel

## 1.2. Extraer coordenadas desde un CSV

In [4]:
import pandas as pd
import os 

#lista para guardar las etiquetas
etiquetas_imged6 = []

#funcion para cargar imagenes, etiquetas y clase
def cargar_data_boxes(_data_annotations, _imagen_name):

    for index, row in _data_annotations.iterrows():
        if row['filename'] == _imagen_name:
            #obtener coordenadas
            clase = row['class']
            xmin = row['xmin']
            ymin = row['ymin']
            xmax = row['xmax'] 
            ymax = row['ymax']
        
            #agregar las coordenadas al array de etiquetas
            etiquetas_imged6.append({
                'class':clase,
                'xmin': xmin,
                'xmax': xmax,
                'ymin': ymin,
                'ymax': ymax
            })

    #aarray de etiquetas
    print(etiquetas_imged6)

data_roboflow = pd.read_csv('data/csv/train/_annotations.csv')#obtener los datos etiquetados de las imagenes originales
#ruta de la imagen descarga despues del etiquetamiento manual
ruta_imagen = "data/csv/train/2024-10-11T17_00_48-809403_1920x1080_0_00_00-000305_jpg.rf.aa51fecf3832c1243c66125c9c2267e2.jpg"
nombre_imagen = os.path.basename(ruta_imagen)#obtener el nombre de la imagen

#obtener la lista de coordenadas de la imagen original
cargar_data_boxes(data_roboflow, nombre_imagen)

[{'class': 'semipellet', 'xmin': 216, 'xmax': 226, 'ymin': 615, 'ymax': 625}, {'class': 'semipellet', 'xmin': 189, 'xmax': 201, 'ymin': 615, 'ymax': 624}, {'class': 'semipellet', 'xmin': 207, 'xmax': 218, 'ymin': 632, 'ymax': 642}, {'class': 'semipellet', 'xmin': 223, 'xmax': 232, 'ymin': 643, 'ymax': 652}, {'class': 'semipellet', 'xmin': 234, 'xmax': 244, 'ymin': 648, 'ymax': 656}, {'class': 'semipellet', 'xmin': 244, 'xmax': 254, 'ymin': 640, 'ymax': 650}, {'class': 'semipellet', 'xmin': 252, 'xmax': 261, 'ymin': 632, 'ymax': 641}, {'class': 'semipellet', 'xmin': 276, 'xmax': 285, 'ymin': 622, 'ymax': 631}, {'class': 'semipellet', 'xmin': 285, 'xmax': 293, 'ymin': 611, 'ymax': 619}, {'class': 'semipellet', 'xmin': 300, 'xmax': 309, 'ymin': 607, 'ymax': 616}, {'class': 'semipellet', 'xmin': 312, 'xmax': 320, 'ymin': 589, 'ymax': 597}, {'class': 'semipellet', 'xmin': 300, 'xmax': 312, 'ymin': 578, 'ymax': 589}, {'class': 'semipellet', 'xmin': 307, 'xmax': 316, 'ymin': 573, 'ymax': 583}

In [2]:
clases = [elemento["class"] for elemento in etiquetas_imged6]
print(clases.count("pellet"))
print(clases.count("semipellet"))

16
9


## 2. Girar plato

In [5]:
import cv2
import numpy as np
import os
import random
import csv

class ImageTagging:
    #función para rotar la imagen sin cambiar el tamaño
    def rotar_imagen_sin_zoom(self, imagen, angulo):
        (h, w) = imagen.shape[:2]
        centro = (w / 2, h / 2)
        M = cv2.getRotationMatrix2D(centro, angulo, 1.0)
        imagen_rotada = cv2.warpAffine(imagen, M, (w, h))
        return imagen_rotada, M

    #función para voltear la imagen horizontalmente
    def voltear_horizontal(self, imagen): 
        return cv2.flip(imagen, 1)  # 1 es para volteo horizontal

    #función para rotar un polígono (caja delimitadora)
    def rotar_poligono(self, puntos, M):
        ones = np.ones(shape=(len(puntos), 1))
        puntos_ones = np.hstack([puntos, ones])
        puntos_rotados = M.dot(puntos_ones.T).T
        return puntos_rotados[:, :2].astype(int)

    #función para convertir las coordenadas de Roboflow a un cuadrilátero
    def convertir_a_cuadrilatero(self, xmin, xmax, ymin, ymax):
        return [
            [xmin, ymin],  # Esquina superior izquierda
            [xmax, ymin],  # Esquina superior derecha
            [xmax, ymax],  # Esquina inferior derecha
            [xmin, ymax]   # Esquina inferior izquierda
        ]

    #función para guardar las etiquetas y las nuevas coordenadas en un archivo CSV
    def guardar_en_csv(self, nombre_imagen, etiquetas, cajas_rotadas, nombre_archivo_csv):
        file_exists = os.path.isfile(nombre_archivo_csv)
        with open(nombre_archivo_csv, mode='a', newline='') as file:
            writer = csv.writer(file)
            #escribir encabezados
            if not file_exists:
                writer.writerow(['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'])
            
            #escribir las etiquetas y las coordenadas de las cajas rotadas
            for etiqueta, caja in zip(etiquetas, cajas_rotadas):
                # Asegurarse de que las cajas estén en el orden correcto y convertirlas a enteros
                xmin = min(caja[:, 0])
                xmax = max(caja[:, 0])
                ymin = min(caja[:, 1])
                ymax = max(caja[:, 1])
                nombre = os.path.basename(nombre_imagen)
                #escribir cada fila en el CSV
                writer.writerow([nombre, "1080","1080", etiqueta['class'], xmin, ymin, xmax, ymax])

#ruta a la imagen (sacar con el formato de roboflow o con el formato de png de la imagen que se carga a roboflow)
#ruta_imagen = "../../ModelCNN/dataset_dag/train/ED6_1_sf1_png.rf.f6149560a84cfdeb4865ff4d061ba347.jpg"
#ruta_imagen = "../../Red1_test/ED6/ED6_1_sf1.png"

IT = ImageTagging()

for i in range(5):
    #el archivo existe
    if not os.path.isfile(ruta_imagen):
        print(f"Error: La imagen no se encuentra en la ruta especificada: {ruta_imagen}")
    else:#leer la imagen
        imagen = cv2.imread(ruta_imagen)
        #por si la imagen es demasiada grande
        if imagen is None:
            print(f"Error: No se pudo cargar la imagen.")
        else:#en el caso de que la imagen es leída correctamente
            print("Imagen cargada correctamente.")

            #etiquetas proporcionadas por Roboflow
            etiquetas = etiquetas_imged6

            #convertir las etiquetas a cuadriláteros
            cajas = [IT.convertir_a_cuadrilatero(etiq['xmin'], etiq['xmax'], etiq['ymin'], etiq['ymax']) for etiq in etiquetas]
            
            #crear una copia de la imagen para dibujar las cajas
            imagen_con_cajas = imagen.copy()
            for caja in cajas:
                puntos = np.array(caja, np.int32)
                puntos = puntos.reshape((-1, 1, 2))
                cv2.polylines(imagen_con_cajas, [puntos], isClosed=True, color=(255, 0, 0), thickness=2)  # Azul para las cajas originales
            
            #definir un rango de ángulos de rotación (-20 a 20 grados)
            angulo = random.uniform(-20, 20)#rotar dentro de este rango
            imagen_rotada, M = IT.rotar_imagen_sin_zoom(imagen, angulo)#controlar que la imagen rote horizontalmente
            print(f"Imagen rotada {angulo:.2f} grados exitosamente.")
            
            #voltear la imagen horizontalmente con 50% de probabilidad
            if random.choice([True, False]):
                imagen_rotada = IT.voltear_horizontal(imagen_rotada)
                print("Imagen volteada horizontalmente.")
            
            #rotar las cajas
            cajas_rotadas = [IT.rotar_poligono(np.array(caja), M) for caja in cajas]
            print("Cajas rotadas exitosamente.")
            
            #dibujar las cajas rotadas en la imagen rotada
            #for caja_rotada in cajas_rotadas:
            #    caja_rotada = caja_rotada.reshape((-1, 1, 2))
            #    cv2.polylines(imagen_rotada, [caja_rotada], isClosed=True, color=(0, 255, 0), thickness=2)  # Verde para las cajas rotadas
            
            #redimensionar imágenes para que tengan la misma altura
            altura_original = imagen_con_cajas.shape[0]
            altura_rotada = imagen_rotada.shape[0]
            if altura_original != altura_rotada:
                ratio = altura_original / altura_rotada
                nuevo_ancho = int(imagen_rotada.shape[1] * ratio)
                imagen_rotada = cv2.resize(imagen_rotada, (nuevo_ancho, altura_original))
                cajas_rotadas = [ (caja_rotada * ratio).astype(int) for caja_rotada in cajas_rotadas]
            
            #concatenar las imágenes horizontalmente
            imagen_comparacion = np.hstack((imagen_con_cajas, imagen_rotada))
            
            #añadir texto para identificar cada imagen
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(imagen_comparacion, 'Original', (10, 30), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(imagen_comparacion, 'Rotada', (imagen_con_cajas.shape[1] + 10, 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
            #Extraer y asignar nombre a las nuevas imagenes
            extraer_nombre1 = nombre_imagen.split('.rf.')[1]#obtengo el codigo de la imagen
            extraer_nombre2 = extraer_nombre1.split('.jpg')[0]#quitar el formato de la imagen
            nombre_imagen_comparacion = f"data/comparacion/F_aug_{extraer_nombre2}_{i+1}.jpg"#guardar para comparar
            nombre_imagen_aumentada = f"data/augmentation/F__aug_{extraer_nombre2}_{i+1}.jpg"#guardar para dataset

            #guardar la imagen comparativa
            cv2.imwrite(nombre_imagen_comparacion, imagen_comparacion)
            cv2.imwrite(nombre_imagen_aumentada, imagen_rotada)

            #después de la rotación y el ajuste de las coordenadas, agregar la función para guardar en CSV
            nombre_archivo_csv = 'data/augmentation/nuevas_coordenadas.csv'
            IT.guardar_en_csv(nombre_imagen_aumentada, etiquetas_imged6, cajas_rotadas, nombre_archivo_csv)
            print(f"Las nuevas coordenadas se han guardado en {nombre_archivo_csv}")
            print(f"Ciclo {i+1} de imagen {nombre_imagen} TERMINADO")

Imagen cargada correctamente.
Imagen rotada -12.36 grados exitosamente.
Imagen volteada horizontalmente.
Cajas rotadas exitosamente.
Las nuevas coordenadas se han guardado en data/augmentation/nuevas_coordenadas.csv
Ciclo 1 de imagen 2024-10-11T17_00_48-809403_1920x1080_0_00_00-000305_jpg.rf.aa51fecf3832c1243c66125c9c2267e2.jpg TERMINADO
Imagen cargada correctamente.
Imagen rotada -11.58 grados exitosamente.
Cajas rotadas exitosamente.
Las nuevas coordenadas se han guardado en data/augmentation/nuevas_coordenadas.csv
Ciclo 2 de imagen 2024-10-11T17_00_48-809403_1920x1080_0_00_00-000305_jpg.rf.aa51fecf3832c1243c66125c9c2267e2.jpg TERMINADO
Imagen cargada correctamente.
Imagen rotada -12.46 grados exitosamente.
Imagen volteada horizontalmente.
Cajas rotadas exitosamente.
Las nuevas coordenadas se han guardado en data/augmentation/nuevas_coordenadas.csv
Ciclo 3 de imagen 2024-10-11T17_00_48-809403_1920x1080_0_00_00-000305_jpg.rf.aa51fecf3832c1243c66125c9c2267e2.jpg TERMINADO
Imagen cargad

In [4]:
nuevas_coordenadas = pd.read_csv("data/augmentation/nuevas_coordenadas.csv")
nuevas_coordenadas["class"].unique()
nuevas_coordenadas["class"].value_counts()

class
pellet        80
semipellet    45
Name: count, dtype: int64

## 3. Unificar dataset

In [5]:
data_roboflow = data_roboflow.reset_index(drop=True)
nuevas_coordenadas = nuevas_coordenadas.reset_index(drop=True)

df_unido = pd.concat([data_roboflow, nuevas_coordenadas], axis=0)
df_unido.to_csv("data/augmentation/df_unido.csv", index=False)

## 4. Graficar etiquetas de la imagen rotada

In [7]:

# Dibujar las cajas rotadas en la imagen rotada y agregar la etiqueta de clase
for etiqueta, caja_rotada in zip(etiquetas_imged6, cajas_rotadas):
    # Dibujar la caja delimitadora
    caja_rotada = caja_rotada.reshape((-1, 1, 2))
    cv2.polylines(imagen_rotada, [caja_rotada], isClosed=True, color=(0, 255, 0), thickness=2)  # Verde para las cajas rotadas

    # Obtener las coordenadas donde se colocará el texto (usar la esquina superior izquierda)
    x_texto, y_texto = caja_rotada[0][0]  # Primera coordenada de la caja rotada (xmin, ymin)
    
    # Agregar la etiqueta de clase sobre la caja
    cv2.putText(imagen_rotada, etiqueta['class'], (x_texto, y_texto - 10), font, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

# Guardar la imagen rotada con las etiquetas
cv2.imwrite('data/augmentation/imagen_rotada_con_etiquetas.jpg', imagen_rotada)
print("Imagen rotada con etiquetas guardada como 'imagen_rotada_con_etiquetas.jpg'")


Imagen rotada con etiquetas guardada como 'imagen_rotada_con_etiquetas.jpg'
